Script para realizar experimentos do tcc

In [1]:
import os
import re
import csv
import math
import numpy as np
import glob

In [2]:
conjunto = "E"
results = "/home/lelis/Documents/Genetic_Algorithms/vrp/results/"
benchmark = "/home/lelis/Documents/Genetic_Algorithms/vrp/benchmarks/"

diretorio = os.path.join(benchmark, conjunto)  # Use o os.path.join para criar o caminho completo
print(diretorio)

if os.path.exists(diretorio):
    files_benchmark = [arquivo for arquivo in os.listdir(diretorio) if arquivo.endswith(".vrp")]
    print(files_benchmark)
else:
    print("O diretório não existe:", diretorio)

/home/lelis/Documents/Genetic_Algorithms/vrp/benchmarks/E
['E-n51-k5.vrp', 'E-n101-k14.vrp', 'E-n23-k3.vrp', 'E-n76-k8.vrp', 'E-n22-k4.vrp', 'E-n101-k8.vrp', 'E-n30-k3.vrp', 'E-n76-k10.vrp', 'E-n76-k7.vrp', 'E-n76-k14.vrp', 'E-n33-k4.vrp']


In [3]:
# Certifique-se de que dataset seja uma string, não um dicionário
if isinstance(conjunto, str):
    caminho_completo = os.path.join(results,conjunto)

    print("Caminho completo:", caminho_completo)

    try:
        os.chdir(caminho_completo)
        print(f"Diretório atual alterado para: {os.getcwd()}")
    except OSError as e:
        print(f"Erro ao alterar o diretório: {e}")
else:
    print("A variável 'dataset' não é uma string.")

Caminho completo: /home/lelis/Documents/Genetic_Algorithms/vrp/results/E
Diretório atual alterado para: /home/lelis/Documents/Genetic_Algorithms/vrp/results/E


In [4]:
files = [arquivo for arquivo in os.listdir() if arquivo.endswith(".out")]
print(files)

['E-n22-k4.out', 'E-n76-k10.out', 'E-n76-k14.out', 'E-n51-k5.out', 'E-n33-k4.out', 'E-n101-k14.out', 'E-n76-k7.out', 'E-n30-k3.out', 'E-n101-k8.out', 'E-n76-k8.out', 'E-n23-k3.out']


In [5]:
results = {"melhor": [float('inf'), None], "pior": [float('-inf'), None], "media": [0, 0]}
saida = open('result.csv', "w", newline='')
writer = csv.writer(saida)
writer.writerow(["Dataset","Melhor Resultado","Tempo",
                 "Resultado Médio","Tempo",
                 "Pior Resultado","Tempo",
                 "Desvio Padrão","Valor Otimo"])


101

In [6]:
def valorOtimo(fileName):
    with open(benchmark + conjunto + '/' + fileName[:-3] + 'vrp', 'r') as arquivo:
        print('Lendo instância: ' + benchmark + conjunto + '/' + fileName[:-3] + 'vrp')
        # Lê o conteúdo do arquivo
        conteudo = arquivo.read()
        
        # Procura por um número após "Optimal value:"
        match = re.search(r'Optimal value:\s*([\d.]+)', conteudo)
        print('Match:' + str(match))

        #verifica se está escrito como 'Best Value'
        if(match is None):
            match = re.search(r'Best value:\s*(\d+)', conteudo)
            print('Match:' + str(match))
        
        # Verifica se houve um match
        if match:
            # O número está no grupo 1 da correspondência (índice 1)
            numero = match.group(1)
            return numero


In [7]:
for file in files:
    values = []
    count = 0
    print("Lendo Resultado " + os.getcwd() + "/" +file)
    try:
        namefile = open(file, mode="r")

        while True:
            count  += 1;
            # Linha com a distância
            distancia = namefile.readline()
            #print(distancia)
            if not distancia:
                break
            distancia = float(distancia)
            values.append(float(distancia))
            

            # Linha com o tempo
            tempo = namefile.readline()
            tempo = re.findall(r'\d+', tempo)
            if not tempo:
                break

            # Atualize os resultados
            if distancia < results['melhor'][0]:
                results['melhor'] = [distancia, tempo]
            elif distancia > results['pior'][0]:
                results['pior'] = [distancia, tempo]

            results['media'] = [results['media'][0] + distancia, results['media'][1] + int(tempo[0])]

    except FileNotFoundError:
        print(f"O arquivo '{file}' não foi encontrado.")
    except Exception as e:
        print(f"Ocorreu um erro ao processar o arquivo '{file}': {str(e)}")
        
    results['media'] = [results['media'][0]/count, results['media'][1]/count]
    
    dataset = re.sub(r'\.out$', '', file)#nome do arquivo sem o '.out'

    np_values = np.array([values])

    print(results, ", Desvio padrão " + str(np_values.std()), valorOtimo(file))

    melhor_tempo_execucao = results['melhor'][1][0] if results['melhor'][1] is not None else ""
    media_tempo_execucao = results['media'][1] if results['media'][1] is not None else ""
    pior_tempo_execucao = results['pior'][1][0] if results['pior'][1] is not None else ""
    
    writer.writerow([dataset, results['melhor'][0], melhor_tempo_execucao, results['media'][0], media_tempo_execucao,
                     results['pior'][0], pior_tempo_execucao, np_values.std(), valorOtimo(file)])
    
    results = {"melhor": [float('inf'), None], "pior": [float('-inf'), None], "media": [0, 0]}
saida.close()

Lendo Resultado /home/lelis/Documents/Genetic_Algorithms/vrp/results/E/E-n22-k4.out
Lendo instância: /home/lelis/Documents/Genetic_Algorithms/vrp/benchmarks/E/E-n22-k4.vrp
Match:<re.Match object; span=(73, 91), match='Optimal value: 375'>
{'melhor': [659.59, ['43']], 'pior': [740.49, ['44']], 'media': [672.0338709677418, 41.16129032258065]} , Desvio padrão 24.23993299633203 375
Lendo instância: /home/lelis/Documents/Genetic_Algorithms/vrp/benchmarks/E/E-n22-k4.vrp
Match:<re.Match object; span=(73, 91), match='Optimal value: 375'>
Lendo Resultado /home/lelis/Documents/Genetic_Algorithms/vrp/results/E/E-n76-k10.out
Lendo instância: /home/lelis/Documents/Genetic_Algorithms/vrp/benchmarks/E/E-n76-k10.vrp
Match:<re.Match object; span=(75, 93), match='Optimal value: 830'>
{'melhor': [2392.33, ['110']], 'pior': [2940.28, ['109']], 'media': [2622.113548387096, 106.48387096774194]} , Desvio padrão 123.90382544358975 830
Lendo instância: /home/lelis/Documents/Genetic_Algorithms/vrp/benchmarks/E/